In [ ]:
%pip install duckdb --quiet

## 📦 Instalação de Dependências

Instale as bibliotecas necessárias para este notebook.

# Capitulo 07 Trabalhando Parquet S3

Notebook gerado automaticamente a partir do código fonte python.
